In [1]:
import hashlib
import random
from math import gcd

# ----------------------------------------
# Utilities
# ----------------------------------------
def mod_hash(*values, q=2**256-189):
    """Hash concatenated values modulo q."""
    h = hashlib.sha256()
    for v in values:
        if isinstance(v, int):
            v = str(v).encode()
        h.update(v)
    return int.from_bytes(h.digest(), "big") % q


# ----------------------------------------
# Setup for Schnorr Protocol
# ----------------------------------------
def schnorr_setup():
    # Use a small safe prime example for demonstration (not secure!)
    # In practice, use large primes such that q | p-1.
    p = 208351617316091241234326746312124448251235562226470491514186331217050270460481
    q = (p - 1) // 2
    g = 2
    return p, q, g


# ----------------------------------------
# Honest Prover & Verifier (Interactive)
# ----------------------------------------
def schnorr_interactive_prover(p, q, g, x):
    """Honest prover knows secret x."""
    r = random.randrange(1, q)
    t = pow(g, r, p)  # commitment
    return r, t

def schnorr_interactive_verifier(p, q, g, y, t):
    """Verifier chooses a random challenge."""
    c = random.randrange(1, q)
    return c

def schnorr_interactive_response(p, q, g, x, r, c):
    """Prover responds."""
    s = (r + c * x) % q
    return s

def schnorr_interactive_verify(p, q, g, y, t, c, s):
    """Verifier checks."""
    lhs = pow(g, s, p)                # g^s
    rhs = (t * pow(y, c, p)) % p      # t * y^c
    return lhs == rhs


# ----------------------------------------
# Cheating Prover (Does NOT know x)
# ----------------------------------------
def cheating_prover_attempt(p, q, g, y):
    """
    Cheating prover tries to guess challenge c in advance.
    Strategy: Pick random (t, c, s) that satisfy g^s = t * y^c mod p.
    This only works if verifier's real c matches the guess.
    """
    c_guess = random.randrange(1, q)
    s = random.randrange(1, q)
    t = (pow(g, s, p) * pow(y, -c_guess, p)) % p  # forge t so equation holds
    return t, c_guess, s


# ----------------------------------------
# Fiat–Shamir Transform (Non-interactive)
# ----------------------------------------
def schnorr_fiat_shamir_prove(p, q, g, x):
    r = random.randrange(1, q)
    t = pow(g, r, p)
    c = mod_hash(str(t).encode(), q=q)  # Hash challenge
    s = (r + c * x) % q
    return (t, s)

def schnorr_fiat_shamir_verify(p, q, g, y, proof):
    t, s = proof
    c = mod_hash(str(t).encode(), q=q)
    lhs = pow(g, s, p)
    rhs = (t * pow(y, c, p)) % p
    return lhs == rhs


# ----------------------------------------
# MAIN EXPERIMENTS
# ----------------------------------------
if __name__ == "__main__":
    p, q, g = schnorr_setup()

    # Secret key x, public key y
    x = random.randrange(1, q)
    y = pow(g, x, p)

    log = []

    # -----------------------------
    # 1. Honest Interactive Run
    # -----------------------------
    r, t = schnorr_interactive_prover(p, q, g, x)
    c = schnorr_interactive_verifier(p, q, g, y, t)
    s = schnorr_interactive_response(p, q, g, x, r, c)
    ok = schnorr_interactive_verify(p, q, g, y, t, c, s)
    log.append(("Honest Interactive", ok))

    # -----------------------------
    # 2. Cheating Prover Experiments
    # -----------------------------
    trials = 500
    success = 0

    for _ in range(trials):
        t_fake, c_guess, s_fake = cheating_prover_attempt(p, q, g, y)
        real_c = schnorr_interactive_verifier(p, q, g, y, t_fake)
        if real_c == c_guess:
            success += schnorr_interactive_verify(p, q, g, y, t_fake, real_c, s_fake)

    cheating_prob = success / trials
    log.append(("Cheating Interactive (Estimated Probability)", cheating_prob))

    # -----------------------------
    # 3. Non-Interactive Fiat–Shamir
    # -----------------------------
    proof = schnorr_fiat_shamir_prove(p, q, g, x)
    ok_noninteractive = schnorr_fiat_shamir_verify(p, q, g, y, proof)
    log.append(("Fiat–Shamir Honest Verification", ok_noninteractive))

    # -----------------------------
    # 4. Cheating in Fiat–Shamir Mode
    # -----------------------------
    # Cheater cannot predict hash(c), so probability ≈ 1/q ~ 0 in practice.
    # But we can simulate attempts with random guesses of t,s.
    fs_trials = 500
    fs_success = 0

    for _ in range(fs_trials):
        # Cheater picks (t, s) but c = H(t) is fixed by hash; can't be chosen.
        t = random.randrange(1, p)
        s = random.randrange(1, q)

        if schnorr_fiat_shamir_verify(p, q, g, y, (t, s)):
            fs_success += 1

    fs_cheating = fs_success / fs_trials
    log.append(("Fiat–Shamir Cheating Probability (Estimated)", fs_cheating))


    # ----------------------------------------
    # Print Results
    # ----------------------------------------
    print("\n=== Schnorr ZKP Experiment Results ===")
    for name, value in log:
        print(f"{name} → {value}")



=== Schnorr ZKP Experiment Results ===
Honest Interactive → True
Cheating Interactive (Estimated Probability) → 0.0
Fiat–Shamir Honest Verification → True
Fiat–Shamir Cheating Probability (Estimated) → 0.0
